In [1]:
class MySVM(BaseEstimator, ClassifierMixin):
    """
    1 vs 1 SVM (binary classification)
    """
    def __init__(self, C=0.1, eta=0.001, batch_size=1, max_iter=25, epsilon=1e-8):
        """
        constructor of MyBinarySVM class.
        """
        
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.eta = eta
        self.C = C
        self.epsilon = epsilon
        self.num_classes = 0
#         self.beta1 = 0.9
#         self.beta2 = 0.99
    
    def fit(self, X, y=None, params=None):
        """
        fit method for training svm
        
        Arguments:
        --------------------------
        X: image data. (60000, 784)
        y: label data. (60000, 1)
        
        Returns:
        --------------------------
        Z: class score
        """

        m = np.shape(X)[0] #행 개수 60000
        n = np.shape(X)[1] #열 개수 784
        self.num_classes = len(np.unique(y)) #클래스 수 = 10
        
        y_encoded = self.encode_y(y)
        
        # create weights.
        if params is None:
            self.params = {
                'W': np.random.randn(n, self.num_classes), #(784,10) 정규분포난수
                'b': np.random.randn(1, self.num_classes)
            }

        cnt = 1
        
        for epoch in range(self.max_iter):
            
            X_shuffled, y_shuffled = self.shuffle(X, y_encoded)

            avg_loss = 0
            
            batch_count = int(np.ceil(np.shape(X)[0] / self.batch_size))
            
            for t in range(batch_count):
                m = np.shape(X_shuffled)[0] #60000
        
        X_batch = X_shuffled[t * self.batch_size : min(m, (t + 1) * self.batch_size)]
        y_batch = y_shuffled[t * self.batch_size : min(m, (t + 1) * self.batch_size)]
        bs = min(m, (t + 1) * self.batch_size) - t * self.batch_size
        
        
                X_batch, y_batch, bs = self.next_batch(X_shuffled, y_shuffled, t)
                
                X_batch = np.reshape(X_batch, (bs, n))
                y_batch = np.reshape(y_batch, (bs, self.num_classes))
                Z = self.forward_prop(X_batch)
                Z = np.reshape(Z, (bs, self.num_classes))
                loss = self.compute_cost(y_batch, Z)
                self.backward_prop(X_batch, y_batch, Z, bs, cnt)
            
                # accumulate loss
                avg_loss += loss
                cnt += 1
        
            # logging
            avg_loss /= batch_count
            if epoch % (self.max_iter / 10) == 0:
                print('Cost at epoch {0}: {1}'.format(epoch, avg_loss))

        return self
    
    def encode_y(self, y):
        y_encoded = np.ones((np.shape(y)[0], self.num_classes)) #1로 이루어진 배열(60000,10)
        
        for i in range(self.num_classes):
            y_encoded[:, i][y != i] = -1
            
        return y_encoded
    
    def backward_prop(self, X, y, Z, bs, cnt):
        """
        update weights
        
        Arguments:
        ----------------------------
        X: images e.g (BATCH_SIZE, 784)
        y: labels e.g (BATCH_SIZE, 1)
        Z: class score after forward propagation
        params: weights dictionary(map in other programming language)
        eta: learning rate
        
        Returns:
        ----------------------------
        params: weights dictionary
        """
        
        # number of features
        n = np.shape(X)[1]
        
        # differential vector of loss function to update weights
        dw = np.zeros(self.params['W'].shape)
        db = np.zeros(self.params['b'].shape)
        
        Z = np.reshape(Z, (bs, self.num_classes))
        temp = np.multiply(y, Z)
        temp = 1 - temp
        
        temp[temp <= 0] = 0
        temp[temp > 0] = 1
        
        y_temp = np.multiply(y, temp.reshape(bs, self.num_classes))
        
        dw = -(1 / bs) * np.matmul(X.T, y_temp) + (1 / self.C) * self.params['W']
        db = -(1 / bs) * np.sum(y_temp, axis=0)

#         if cnt == 1:
#             self.M['W'] = dw
#             self.M['b'] = db
#         else:
#             self.M['W'] = (self.beta1 * self.M['W'] + (1 - self.beta1) * dw)
#             self.M['b'] = (self.beta1 * self.M['b'] + (1 - self.beta1) * db)
        
#         if cnt == 1:
#             self.V['W'] = dw ** 2
#             self.V['b'] = db ** 2
#         else:
#             self.V['W'] = (self.beta2 * self.V['W'] + (1 - self.beta2) * (dw ** 2))
#             self.V['b'] = (self.beta2 * self.V['b'] + (1 - self.beta2) * (db ** 2))
    
#         self.M['W'] = (self.beta1 * self.M['W'] + (1 - self.beta1) * dw) / (1 - self.beta1 ** cnt)
#         self.M['b'] = (self.beta1 * self.M['b'] + (1 - self.beta1) * db) / (1 - self.beta1 ** cnt)
        
#         self.V['W'] = (self.beta2 * self.V['W'] + (1 - self.beta2) * np.square(dw)) / (1 - self.beta2 ** cnt)
#         self.V['b'] = (self.beta2 * self.V['b'] + (1 - self.beta2) * np.square(db)) / (1 - self.beta2 ** cnt)

        # update weights
#         self.params['W'] = self.params['W'] - np.divide(self.eta * self.M['W'], np.sqrt(self.V['W']) + self.epsilon)
#         self.params['b'] = self.params['b'] - np.divide(self.eta * self.M['b'], np.sqrt(self.V['b']) + self.epsilon)
        
        self.params['W'] = self.params['W'] - (self.eta / (1 + self.epsilon * cnt)) * dw
        self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * db

#         self.params['W'] = self.params['W'] - (self.eta / (1 + self.epsilon * cnt)) * dw
#         self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * db
        
        return self.params
    
    def predict(self, X, y=None):
        m = np.shape(X)[0]
        
        class_score = self.forward_prop(X)
        pred = np.argmax(class_score, axis=1)
        
        return pred
    
    def score(self, X, y=None):
        pred = self.predict(X)
        score = np.mean(pred == y)
        
        return score
    
    def get_parameters(self):
        return self.params

NameError: name 'BaseEstimator' is not defined